<a href="https://colab.research.google.com/github/olonok69/LLM_Notebooks/blob/main/t5/T5_sequence_to_sequence_custom_mlflow_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MLFLOW
https://mlflow.org/docs/latest/introduction/index.html


MLflow is a solution to many of these issues in this dynamic landscape, offering tools and simplifying processes to streamline the ML lifecycle and foster collaboration among ML practitioners.

https://mlflow.org/docs/latest/llms/llm-evaluate/index.html

# MLflow transformers Guide
https://mlflow.org/docs/latest/llms/transformers/guide/index.html
https://mlflow.org/docs/latest/llms/transformers/tutorials/fine-tuning/transformers-fine-tuning.html


# ngrok
Connect localhost to the internet for testing applications and APIs
Bring secure connectivity to apps and APIs in localhost and dev/test environments with just one command or function call.
- Webhook testing
- Developer Previews
- Mobile backend testing

https://ngrok.com/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mlflow pyngrok evaluate  bitsandbytes accelerate datasets transformers==4.39.3 --quiet
get_ipython().system_raw("mlflow ui --port 5000 &")

In [ ]:

from pyngrok import ngrok
from getpass import getpass

# Terminate open tunnels if exist
ngrok.kill()

In [ ]:
from google.colab import userdata
NGROK_AUTH_TOKEN  = userdata.get('NGROK')

ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

MLflow Tracking UI: https://4566-34-125-199-93.ngrok-free.app


## Fine-Tuning Transformers with MLflow for Enhanced Model Management



In [ ]:
# Disable tokenizers warnings when constructing pipelines
%env TOKENIZERS_PARALLELISM=false

import warnings

# Disable a few less-than-useful UserWarnings from setuptools and pydantic
warnings.filterwarnings("ignore", category=UserWarning)

env: TOKENIZERS_PARALLELISM=false


### Preparing the Dataset and Environment for Fine-Tuning

#### Key Steps in this Section

1. **Loading the Dataset**: Utilizing the `sms_spam` dataset for spam detection.
2. **Splitting the Dataset**: Dividing the dataset into training and test sets with an 80/20 distribution.
3. **Importing Necessary Libraries**: Including libraries like `evaluate`, `mlflow`, `numpy`, and essential components from the `transformers` library.

Before diving into the fine-tuning process, setting up our environment and preparing the dataset is crucial. This step involves loading the dataset, splitting it into training and testing sets, and initializing essential components of the Transformers library. These preparatory steps lay the groundwork for an efficient fine-tuning process.

This setup ensures that we have a solid foundation for fine-tuning our model, with all the necessary data and tools at our disposal. In the following Python code, we'll execute these steps to kickstart our model fine-tuning journey.

In [ ]:
import evaluate
import numpy as np
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    pipeline,
)

import mlflow
import pandas as pd
# Load the "sms_spam" dataset.
# sms_dataset = load_dataset("sms_spam")

# # Split train/test by an 8/2 ratio.
# sms_train_test = sms_dataset["train"].train_test_split(test_size=0.2)
# train_dataset = sms_train_test["train"]
# test_dataset = sms_train_test["test"]

In [ ]:
data_path = "/content/drive/MyDrive/data/documents_final_cv.csv"
data = pd.read_csv(data_path, header=None)
data_path_test = "/content/drive/MyDrive/data/documents_test_cv.csv"
data_test= pd.read_csv(data_path_test,  header=None)
data.columns = ["label", "text"]
data_test.columns = ["label", "text"]

In [ ]:
#label_dic = {"cv": 1, "non-cv": 0}

In [ ]:
# data['label'] = data['label'].map(label_dic)
# data_test['label'] = data_test['label'].map(label_dic)

In [ ]:
df_train_test = Dataset.from_pandas(data)
df_val = Dataset.from_pandas(data_test)
df_train_test = df_train_test.train_test_split(test_size=0.2)

In [ ]:
train_dataset = df_train_test["train"]
test_dataset = df_train_test["test"]

In [ ]:
train_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 160
})

In [ ]:
test_dataset

Dataset({
    features: ['label', 'text'],
    num_rows: 40
})

In [ ]:
train_dataset[0]['text']

'evaluation document created fernanda date donec diam vestibulum vulputate ultrices vestibulum ante ipsum primis faucibus orci luctus et ultrices posuere cubilia donec magna vestibulum aliquet erat tortor sollicitudin sit amet lobortis sapien sapien non integer ac morbi non quam nec dui luctus nulla curabitur gravida nisi hac habitasse platea aliquam augue sollicitudin consectetuer rutrum sed vivamus duis mattis egestas proin eu nulla ac turpis nec euismod quam turpis adipiscing vitae mattis nibh ligula nec aliquam convallis proin turpis pede posuere integer non diam vestibulum vulputate ultrices vestibulum ante ipsum primis faucibus orci luctus et ultrices posuere cubilia donec magna vestibulum aliquet erat tortor sollicitudin sit amet lobortis sapien sapien non integer ac maecenas ut massa quis augue luctus nulla mollis molestie quisque ut exam proin interdum mauris non ligula pellentesque phasellus id sapien sapien iaculis vivamus metus adipiscing hendrerit vulputate pellentesque eg

In [ ]:
train_dataset[0]['label']

'non-cv'

In [ ]:
import evaluate
import nltk
import numpy as np
from typing import List, Tuple
from nltk.tokenize import sent_tokenize
from datasets import Dataset, concatenate_datasets
from huggingface_hub import HfFolder
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)

In [ ]:
PROJECT = "FlanT5-Custom"
MODEL_NAME = 'google/flan-t5-base'
DATASET = "CVS-Premcloud"

In [ ]:
MODEL_ID = "google/flan-t5-base"
# Load tokenizer of FLAN-t5
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [ ]:
# Metric
metric = evaluate.load("f1")
metric1 = evaluate.load("roc_auc")
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([train_dataset, test_dataset]).map(
    lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label']
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Max source length: 512


In [ ]:
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([train_dataset, test_dataset]).map(
    lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label']
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Max target length: 5


In [ ]:
REPOSITORY_ID = f"{MODEL_ID.split('/')[1]}-text-classification"
REPOSITORY_ID

'flan-t5-base-text-classification'

### Model Initialization and Label Mapping

Next, we'll set up label mappings and initialize the model for our text classification task.

Having prepared our data, the next crucial step is to initialize our model and set up label mappings. This involves defining a clear relationship between the labels in our dataset and their corresponding representations in the model.

#### Setting Up Label Mappings

- **Defining Label Mappings**: Creating bi-directional mappings between integer labels and textual representations ("ham" and "spam").

#### Initializing the Model

- **Model Selection**: Choosing the `distilbert-base-uncased` model for its balance of performance and efficiency.
- **Model Configuration**: Configuring the model for sequence classification with the defined label mappings.

Proper model initialization and label mapping are key to ensuring that the model accurately understands and processes the task at hand. By explicitly defining these mappings and selecting an appropriate pre-trained model, we lay the groundwork for effective and efficient fine-tuning.

In [ ]:
# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=REPOSITORY_ID,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False,     # Overflows with fp16
    learning_rate=3e-4,
    num_train_epochs=4,
    logging_dir=f"{REPOSITORY_ID}/logs",    # logging & evaluation strategies
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,

)

def preprocess_function(sample: Dataset, padding: str = "max_length") -> dict:
    """ Preprocess the dataset. """

    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


def postprocess_text(preds: List[str], labels: List[str]) -> Tuple[List[str], List[str]]:
    """ helper function to postprocess text"""
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    # load metrics
    metric = evaluate.load("f1")
    metric1 = evaluate.load("roc_auc")
    metric2 = evaluate.load("recall")
    metric3 = evaluate.load("precision")

    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    id2label = {0: "non-cv", 1: "cv"}
    label2id = {"non-cv": 0, "cv": 1}

    decoded_preds_bin = np.array([label2id.get(x) for x in decoded_preds ])
    decoded_labels_bin = np.array([label2id.get(x) for x in decoded_labels ])
    result = metric.compute(predictions=decoded_preds_bin, references=decoded_labels_bin, average='macro')
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result['roc_auc'] = metric1.compute(prediction_scores=decoded_preds_bin, references=decoded_labels_bin, average='macro')['roc_auc']
    result['recall'] = metric2.compute(predictions=decoded_preds_bin, references=decoded_labels_bin, average='macro')['recall']
    result['precision'] = metric3.compute(predictions=decoded_preds_bin, references=decoded_labels_bin, average='macro')['precision']

    return result

In [ ]:
tokenized_dataset = df_train_test.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


# model loader with a sequence-to-sequence language modeling head
https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSeq2SeqLM

In [ ]:
# Set the mapping between int label and its meaning.
id2label = {0: "non-cv", 1: "cv"}
label2id = {"non-cv": 0, "cv": 1}

In [ ]:
# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID,    num_labels=2,
    label2id=label2id,
    id2label=id2label,)

In [ ]:
nltk.download("punkt")

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(



### Configuring the Training Environment

In this step, we're going to configure our Trainer, supplying important training configurations via the use of the `TrainingArguments` API.

With our model and metrics ready, the next important step is to configure the training environment. This involves setting up the training arguments and initializing the Trainer, a component that orchestrates the model training process.

#### Training Arguments Configuration

- **Defining the Output Directory**: We specify the `training_output_dir` where our model checkpoints will be saved during training. This helps in managing and storing model states at different stages of training.
- **Specifying Training Arguments**: We create an instance of `TrainingArguments` to define various parameters for training, such as the output directory, evaluation strategy, batch sizes for training and evaluation, logging frequency, and the number of training epochs. These parameters are critical for controlling how the model is trained and evaluated.

#### Initializing the Trainer

- **Creating the Trainer Instance**: We use the Trainer class from the Transformers library, providing it with our model, the previously defined training arguments, datasets for training and evaluation, and the function to compute metrics.
- **Role of the Trainer**: The Trainer handles all aspects of training and evaluating the model, including the execution of training loops, handling of data batching, and calling the compute metrics function. It simplifies the training process, making it more streamlined and efficient.

####  Importance of Proper Training Configuration
Setting up the training environment correctly is essential for effective model training. Proper configuration ensures that the model is trained under optimal conditions, leading to better performance and more reliable results.

In the following code block, we'll configure our training environment and initialize the Trainer, setting the stage for the actual training process.

In [ ]:
# If you are running this tutorial in local mode, leave the next line commented out.
# Otherwise, uncomment the following line and set your tracking uri to your local or remote tracking server.

mlflow.set_tracking_uri("http://127.0.0.1:5000")

### Integrating MLflow for Experiment Tracking

The final preparatory step before beginning the training process is to integrate MLflow for experiment tracking.
   
MLflow is a critical tool in our workflow, enabling us to log, monitor, and compare different runs of our model training.

#### Setting up the MLflow Experiment

- **Naming the Experiment**: We use `mlflow.set_experiment` to create a new experiment or assign the current run to an existing experiment. In this case, we name our experiment "Spam Classifier Training". This name should be descriptive and related to the task at hand, aiding in organizing and identifying experiments later.
- **Role of MLflow in Training**: By setting up an MLflow experiment, we can track various aspects of our model training, such as parameters, metrics, and outputs. This tracking is invaluable for comparing different models, tuning hyperparameters, and maintaining a record of our experiments.

#### Benefits of Experiment Tracking
Utilizing MLflow for experiment tracking offers several advantages:

- **Organization**: Keeps your training runs organized and easily accessible.
- **Comparability**: Allows for easy comparison of different training runs to understand the impact of changes in parameters or data.
- **Reproducibility**: Enhances the reproducibility of experiments by logging all necessary details.

With MLflow set up, we're now ready to begin the training process, keeping track of every important aspect along the way.

In the next code snippet, we'll set up our MLflow experiment for tracking the training of our spam classification model.

In [ ]:
# Pick a name that you like and reflects the nature of the runs that you will be recording to the experiment.
mlflow.set_experiment("T5 Custom Classifier Training")

<Experiment: artifact_location='mlflow-artifacts:/339305791644177581', creation_time=1714495559798, experiment_id='339305791644177581', last_update_time=1714495559798, lifecycle_stage='active', name='T5 Custom Classifier Training', tags={}>

### Starting the Training Process with MLflow

In this step, we initiate the fine-tuning training run, utilizing the native auto-logging functionality to record the parameters used and loss metrics calculated during the training process.
    
With our model, training arguments, and MLflow experiment set up, we are now ready to start the actual training process. This step involves initiating an MLflow run, which will encapsulate all the training activities and metrics.

#### Initiating the MLflow Run

- **Starting an MLflow Run**: We use `mlflow.start_run()` to begin a new MLflow run. This function creates a new run context, under which all the training operations and logging will occur.
- **Training the Model**: Inside the MLflow run context, we call `trainer.train()` to start training our model. This function will run the training loop, processing the data in batches, updating model parameters, and evaluating the model.

#### Monitoring the Training Progress
During training, the `Trainer` object will output logs that provide valuable insights into the training progress:

- **Loss**: Indicates the model's performance, with lower values signifying better performance.
- **Learning Rate**: Shows the current learning rate used during training.
- **Epoch Progress**: Displays the progress through the current epoch.

These logs are crucial for monitoring the model's learning process and making any necessary adjustments. By tracking these metrics within an MLflow run, we can maintain a comprehensive record of the training process, enhancing reproducibility and analysis.

In the next code block, we will start our MLflow run and begin training our model, closely observing the output to gauge the training progress.

In [ ]:
!rm -rf ./flan-T5-fine-tune

In [ ]:
!mkdir ./flan-T5-fine-tune
custom_path = "./flan-T5-fine-tune/"

In [ ]:
with mlflow.start_run() as run:
    train_results = trainer.train()
    print(train_results.metrics)
    trainer.model.save_pretrained(custom_path)
    trainer.data_collator.tokenizer.save_pretrained(custom_path)

    transformers_model = {"model": trainer.model, "tokenizer": trainer.data_collator.tokenizer}
    task = "text-classification"
    model_info = mlflow.transformers.log_model(
        transformers_model=transformers_model,
        artifact_path="text_classifier",
        task=task,
    )
    print(model_info)


Epoch,Training Loss,Validation Loss,F1,Gen Len,Roc Auc,Recall,Precision
1,0.493200,0.000020,1.000000,4.575000,1.000000,1.000000,1.000000
2,0.007900,0.094425,0.974603,4.550000,0.978261,0.978261,0.972222
3,0.005200,0.067933,0.974603,4.550000,0.978261,0.978261,0.972222
4,0.000000,0.036313,0.974603,4.550000,0.978261,0.978261,0.972222


{'train_runtime': 73.4836, 'train_samples_per_second': 8.709, 'train_steps_per_second': 1.089, 'total_flos': 438244705566720.0, 'train_loss': 0.12657344202234527, 'epoch': 4.0}


The model 'T5ForConditionalGeneration' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'GPT2F

In [ ]:
run.to_dictionary()

{'info': {'artifact_uri': 'mlflow-artifacts:/339305791644177581/0a3ff30f60d943d988d218c74908cddc/artifacts',
  'end_time': None,
  'experiment_id': '339305791644177581',
  'lifecycle_stage': 'active',
  'run_id': '0a3ff30f60d943d988d218c74908cddc',
  'run_name': 'resilient-mouse-658',
  'run_uuid': '0a3ff30f60d943d988d218c74908cddc',
  'start_time': 1714502609621,
  'status': 'RUNNING',
  'user_id': 'root'},
 'data': {'metrics': {},
  'params': {},
  'tags': {'mlflow.user': 'root',
   'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
   'mlflow.source.type': 'LOCAL',
   'mlflow.runName': 'resilient-mouse-658'}}}

In [ ]:
run.data

<RunData: metrics={}, params={}, tags={'mlflow.runName': 'resilient-mouse-658',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>

In [ ]:
import transformers
from mlflow.models import infer_signature
from mlflow.transformers import generate_signature_output
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
classification_components = mlflow.transformers.load_model(
    model_info.model_uri, return_type="components"
)


2024/04/30 18:46:36 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false
2024/04/30 18:46:48 INFO mlflow.transformers: 'runs:/0a3ff30f60d943d988d218c74908cddc/text_classifier' resolved as 'mlflow-artifacts:/339305791644177581/0a3ff30f60d943d988d218c74908cddc/artifacts/text_classifier'


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Loading and Testing the Model from MLflow

After logging our fine-tuned model to MLflow, we'll now load and test it.
    
#### Loading the Model from MLflow

- **Using mlflow.transformers.load_model**: We use this function to load the model stored in MLflow. This demonstrates how models can be retrieved and utilized post-training, ensuring they are accessible for future use.
- **Retrieving Model URI**: We use the `model_uri` obtained from logging the model to MLflow. This URI is the unique identifier for our logged model, allowing us to retrieve it accurately.

#### Testing the Model with Validation Text

- **Preparing Validation Text**: We use a creatively crafted text to test the model's performance. This text is designed to mimic a typical spam message, which is relevant to our model's training on spam classification.
- **Evaluating Model Output**: By passing this text through the loaded model, we can observe its performance and effectiveness in a practical scenario. This step is crucial to ensure that the model works as expected in real-world conditions.

Testing the model after loading it from MLflow is essential for several reasons:

- **Validation of Logging Process**: It confirms that the model was logged and loaded correctly.
- **Practical Performance Assessment**: Provides a real-world assessment of the model's performance, which is critical for deployment decisions.
- **Demonstrating End-to-End Workflow**: Showcases a complete workflow from training, logging, loading, to using the model, which is vital for understanding the entire model lifecycle.

In the next code block, we'll load our model from MLflow and test it with a validation text to assess its real-world performance.

In [ ]:
def load_dataset_test(data_path) -> Dataset:
    """ Load dataset. """
    stratify_column_name = "label2"
    dataset_ecommerce_pandas = pd.read_csv(data_path, header=None, names=['label', 'text'])
    dataset_ecommerce_pandas['label2']= dataset_ecommerce_pandas['label'].values
    dataset_ecommerce_pandas['label'] = dataset_ecommerce_pandas['label'].astype(str)
    dataset_ecommerce_pandas['label2'] = dataset_ecommerce_pandas['label2'].astype(str)
    dataset_ecommerce_pandas['text'] = dataset_ecommerce_pandas['text'].astype(str)
    dataset = Dataset.from_pandas(dataset_ecommerce_pandas)

    return dataset

In [ ]:
datatest= load_dataset_test(data_path_test)

In [ ]:
len(datatest)

284

In [ ]:
datatest[0]

{'label': 'cv',
 'text': 'justin wa adkins thrives cultivating business workflows creating strategic key stakeholders simple effective delights customers key inflection points inside business focusing innovation bringing 8 years coding experience managing mission world better place supporting make central actuarial science may manager 2021 construction accounting consulting telecom erp related project data conversion ms dynamics solomon vista 3 months limited solomon cio role managing multiple systems bringing cohesion lead senior analyst 2016 april systems construction accounting battle erp related project project code vista etl modules reconcile jc beginning balances using managed implemented ms materials module batch creation multiple streamlined xml certified payroll report efficient washington equipment gps import allocation procedure built forecast software project managers upload capabilities directly viewpoint database lead eos level 10 weekly meetings mentored 6 various busine

In [ ]:
import torch
from tqdm.auto import tqdm

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import classification_report

In [ ]:
model = classification_components.get('model')
tokenizer = classification_components.get('tokenizer')

In [ ]:
def classify(text_to_classify: str) -> str:
    """Classify a text using the model."""
    inputs = tokenizer.encode_plus(text_to_classify, padding='max_length', max_length=512, return_tensors='pt')
    inputs = inputs.to('cuda') if torch.cuda.is_available() else inputs.to('cpu')
    outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], max_length=5)

    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return prediction


def evaluate_model() -> None:
    """Evaluate the model on the test dataset."""
    predictions_list, labels_list = [], []

    samples_number = len(datatest)
    progress_bar = tqdm(range(samples_number))

    for i in range(samples_number):
        text = datatest['text'][i]
        predictions_list.append(classify(text))
        labels_list.append(str(datatest['label'][i]))

        progress_bar.update(1)

    report = classification_report(labels_list, predictions_list)
    print(report)

In [ ]:
evaluate_model()

  0%|          | 0/284 [00:00<?, ?it/s]

              precision    recall  f1-score   support

          cv       0.97      1.00      0.99       142
      non-cv       1.00      0.97      0.99       142

    accuracy                           0.99       284
   macro avg       0.99      0.99      0.99       284
weighted avg       0.99      0.99      0.99       284



In [ ]:
ngrok.kill()